<a href="https://colab.research.google.com/github/ChaithanyaSaiB/UMBC-DATA606-Capstone/blob/main/notebooks/Dataset_Creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Importing necessary libraries
import pandas as pd
import re
import os
import requests
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
# Import the tqdm library for progress bar
from tqdm import tqdm

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# Load CNN health data
cnnhealth = pd.read_csv("/content/cnnhealth.txt", sep=r"(?<!\s)[|](?!\s)", names=['ID', 'DateTime', 'Article Title and Link'], engine="python")
cnnhealth.head()

,ID,DateTime,Article Title and Link
0,576880531301801984,Sat Mar 14 23:00:11 +0000 2015,An abundance of online info can turn us into e...
1,576820122666471424,Sat Mar 14 19:00:08 +0000 2015,A plant-based diet that incorporates fish may ...
2,576744652717461504,Sat Mar 14 14:00:15 +0000 2015,It doesn't take much to damage your hearing at...
3,576736754436304896,Sat Mar 14 13:28:52 +0000 2015,RT @CNN: Forever young? Discover this island’s...
4,576736614766010368,Sat Mar 14 13:28:18 +0000 2015,RT @CNN: Is post-traumatic stress disorder in ...


In [ ]:
# Extract all URLs from article titles and links
cnnhealth['URL'] = [re.findall(r'https?://\S+', text) for text in cnnhealth['Article Title and Link']]

# Drop rows with multiple or no URLs
drop_indices = cnnhealth['Article Title and Link'][cnnhealth['URL'].apply(len) != 1].index
cnnhealth.drop(drop_indices, inplace=True)

# Extract CNN URL for articles
cnnhealth['URL'] = [url[0] for url in cnnhealth['URL']]

In [ ]:
# Function to fetch content from a URL
def fetch_content(url):
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            return response.text
        else:
            return None
    except Exception as e:
        print("Error fetching content:", e)
        return None

# Function to extract body content from HTML using BeautifulSoup
def extract_body(html_content):
    if html_content:
        soup = BeautifulSoup(html_content, 'html.parser')
        article_body = soup.findAll('p', class_='paragraph')
        if article_body:
            return article_body
        else:
            return None
    else:
        return None

# Apply web scraping to extract content and store it in a new column
def extract_content_with_progress(url_list):
    content_list = []
    for url in tqdm(url_list, desc='Extracting content'):
        content = extract_body(fetch_content(url))
        content_list.append(content)
    return content_list

# Apply web scraping to extract content and store it in a new column
cnnhealth['Content'] = extract_content_with_progress(cnnhealth['URL'])

Extracting content:   0%|          | 15/3299 [00:14<40:11,  1.36it/s]/usr/lib/python3.10/html/parser.py:170: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)
Extracting content:   3%|▎         | 113/3299 [01:15<27:07,  1.96it/s]

Error fetching content: Failed to parse: https://t…


Extracting content:   6%|▌         | 200/3299 [02:04<41:49,  1.23it/s]

Error fetching content: Failed to parse: http://t.c…


Extracting content:   7%|▋         | 246/3299 [02:29<24:43,  2.06it/s]

...

Error fetching content: HTTPConnectionPool(host='t.', port=80): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x7b450f139e40>: Failed to resolve 't' ([Errno -2] Name or service not known)"))


Extracting content:  81%|████████  | 2662/3299 [26:34<00:13, 47.87it/s]

Error fetching content: HTTPConnectionPool(host='t.c', port=80): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x7b450f13b610>: Failed to resolve 't.c' ([Errno -2] Name or service not known)"))


Extracting content: 100%|██████████| 3299/3299 [26:55<00:00,  2.04it/s]

Error fetching content: HTTPConnectionPool(host='on.cnn.com', port=80): Max retries exceeded with url: /SO1PxT (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7b450dbd85b0>, 'Connection to on.cnn.com timed out. (connect timeout=10)'))


In [ ]:
# Drop rows with missing content
cnnhealth.dropna(subset=['Content'], inplace=True)

# Combine content paragraphs into a single string
cnnhealth['Content'] = [" ".join([p_tag.text.strip() for p_tag in content]) for content in cnnhealth['Content']]

In [ ]:
# Clean the content data
def clean_text(text):
    text = re.sub(r'-', ' ', text)  # Substitute hyphens with empty spaces
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove non-alphabetic characters
    text = text.lower()  # Convert text to lowercase
    return text

cnnhealth['Content'] = cnnhealth['Content'].apply(clean_text)

In [ ]:
# Convert to wordnet tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None  # Use default POS for lemmatization

In [ ]:
# Tokenize and preprocess text data
def preprocess_text(text):
    tokens = word_tokenize(text)  # Tokenize text
    lemmatizer = WordNetLemmatizer()  # Initialize lemmatizer
    pos_tags = nltk.pos_tag(tokens)  # Get part-of-speech tags
    for i, (token, tag) in enumerate(pos_tags):
        pos = get_wordnet_pos(tag)  # Convert NLTK POS tags to WordNet POS tags
        if pos:
            tokens[i] = lemmatizer.lemmatize(token, pos=pos)  # Lemmatize tokens
        else:
            tokens[i] = lemmatizer.lemmatize(token)  # Use default POS for lemmatization
    stop_words = set(stopwords.words('english'))  # Get stopwords
    custom_stopwords = [    # Custom stopwords
    "patient", "doctor", "say", "year", "state", "day", "need", "come", "well",
    "make", "think", "know", "go", "use", "one", "like", "people", "may",
    "many", "still", "even", "two", "way", "good", "much", "back", "new",
    "time", "first", "really",
    "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n",
    "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z"
    ]
    tokens = [token for token in tokens if token not in stop_words and token not in custom_stopwords]  # Remove stopwords
    return ' '.join(tokens)

cnnhealth['Content'] = cnnhealth['Content'].apply(preprocess_text)

In [ ]:
# Save preprocessed data to CSV
cnnhealth.to_csv('cnnhealth_preprocessed.csv', index=False)

In [2]:
# Define the directory path containing the .txt files
directory = '/content'

# Initialize an empty list to store DataFrames
dfs = []

# Iterate over all files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        # Read the .txt file into a DataFrame
        df_name = os.path.splitext(filename)[0]  # Extract filename without extension
        try:
            df = pd.read_csv(os.path.join(directory, filename),
                             sep=r"(?<!\s)[|](?!\s)",
                             names=['ID', 'DateTime', 'Article Title and Link'],
                             engine="python")
            # Add the DataFrame to the list
            dfs.append(df)
            print(f"DataFrame '{df_name}' loaded with {len(df)} rows.")
        except UnicodeDecodeError:
            print(f"Error reading {filename}: UnicodeDecodeError. Skipping this file.")

# Concatenate all DataFrames in the list into one DataFrame
combined_df = pd.concat(dfs, ignore_index=True)

# Display the combined DataFrame
print("Combined DataFrame:")
print(combined_df)

DataFrame 'everydayhealth' loaded with 3239 rows.
DataFrame 'nprhealth' loaded with 4837 rows.
DataFrame 'cnnhealth' loaded with 4061 rows.
DataFrame 'cbchealth' loaded with 3741 rows.
DataFrame 'latimeshealth' loaded with 4171 rows.
Combined DataFrame:
                       ID                        DateTime  \
0      304596701757464576  Thu Feb 21 14:21:27 +0000 2013   
1      304595191329853441  Thu Feb 21 14:15:27 +0000 2013   
2      304587659018371072  Thu Feb 21 13:45:31 +0000 2013   
3      304580073380524032  Thu Feb 21 13:15:22 +0000 2013   
4      304572560270573569  Thu Feb 21 12:45:31 +0000 2013   
...                   ...                             ...   
20044  144952345170690052  Fri Dec 09 01:31:46 +0000 2011   
20045  144936916838649857  Fri Dec 09 00:30:28 +0000 2011   
20046  144917037414756354  Thu Dec 08 23:11:28 +0000 2011   
20047  144881597869592576  Thu Dec 08 20:50:39 +0000 2011   
20048  144861369597050880  Thu Dec 08 19:30:16 +0000 2011   

             

In [3]:
# Extract all URLs from article titles and links in combined_df
combined_df['URL'] = [re.findall(r'https?://\S+', text) for text in combined_df['Article Title and Link']]

# Drop rows with multiple or no URLs
drop_indices = combined_df['Article Title and Link'][combined_df['URL'].apply(len) != 1].index
combined_df.drop(drop_indices, inplace=True)

# Extract URL for articles
combined_df['URL'] = [url[0] for url in combined_df['URL']]


In [57]:
!pip install fake-useragent

In [5]:
# Calculate the total number of rows
total_rows = len(combined_df)

# Calculate the size of each split
split_size = total_rows // 10

# Define the start and end indices for each split
split_1 = combined_df.iloc[:split_size].copy()
split_2 = combined_df.iloc[split_size:2*split_size].copy()
split_3 = combined_df.iloc[2*split_size:3*split_size].copy()
split_4 = combined_df.iloc[3*split_size:4*split_size].copy()
split_5 = combined_df.iloc[4*split_size:5*split_size].copy()
split_6 = combined_df.iloc[5*split_size:6*split_size].copy()
split_7 = combined_df.iloc[6*split_size:7*split_size].copy()
split_8 = combined_df.iloc[7*split_size:8*split_size].copy()
split_9 = combined_df.iloc[8*split_size:9*split_size].copy()
split_10 = combined_df.iloc[9*split_size:].copy()

In [62]:
import requests
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
from urllib.parse import urlparse

# Create a UserAgent instance
ua = UserAgent()

# Function to fetch content from a URL
def fetch_content(url):
    try:
        # Define user-agent header using a random user agent
        headers = {'User-Agent': ua.random}

        # Create a session to handle cookies and maintain connection state
        with requests.Session() as session:
            response = session.get(url, headers=headers, timeout=4, allow_redirects=True)
            response.raise_for_status()  # Raise an error for 4xx and 5xx status codes
            final_url = response.url  # Get the final URL after following redirects
            website_name = get_website_name(final_url)
            return (response.text, website_name)
    except Exception as e:
        print("Error fetching content:", e)
        return (None, None)

# Function to extract website name from URL
def get_website_name(url):
    parsed_url = urlparse(url)
    return parsed_url.netloc

# Function to extract body content from HTML using BeautifulSoup
def extract_body(html_content_with_website_name):
    html_content, website_name = html_content_with_website_name
    if html_content:
        soup = BeautifulSoup(html_content, 'html.parser')
        if soup:
            if website_name == 'www.cnn.com':
                article_body = soup.findAll('p', class_='paragraph')
            elif website_name == 'www.health.usnews.com':
                article_body = soup.findAll('p')
            elif website_name == 'www.latimes.com':
                article_body = soup.findAll('p')
            elif website_name == 'www.npr.org':
                storytext_div = soup.find('div', id='storytext')
                if storytext_div:
                    article_body = storytext_div.find_all('p')
                else:
                    article_body = None
            elif website_name == 'www.cbc.ca':
                storytext_div = soup.find('div', class_='story')
                if storytext_div:
                    article_body = storytext_div.find_all('p')
                else:
                    article_body = None
            else:
                article_body = None

            if article_body:
                return article_body
            else:
                return None
        else:
            return None
    else:
        return None

# Apply web scraping to extract content and store it in a new column
def extract_content_with_progress(url_list):
    content_list = []
    for url in tqdm(url_list, desc='Extracting content'):
        content = extract_body(fetch_content(url))
        content_list.append(content)
    return content_list

# Apply web scraping to extract content and store it in a new column
#combined_df['Content'] = extract_content_with_progress(combined_df['URL'])

In [ ]:
split_1['Content'] = extract_content_with_progress(split_1['URL'])
split_1.to_csv('split_1_preprocessed.csv', index=False)

In [ ]:
split_2['Content'] = extract_content_with_progress(split_2['URL'])
split_2.to_csv('split_2_preprocessed.csv', index=False)

In [ ]:
split_3['Content'] = extract_content_with_progress(split_3['URL'])
split_3.to_csv('split_3_preprocessed.csv', index=False)

In [ ]:
split_4['Content'] = extract_content_with_progress(split_4['URL'])
split_4.to_csv('split_4_preprocessed.csv', index=False)

In [ ]:
split_5['Content'] = extract_content_with_progress(split_5['URL'])
split_5.to_csv('split_5_preprocessed.csv', index=False)

In [ ]:
split_6['Content'] = extract_content_with_progress(split_6['URL'])
split_6.to_csv('split_6_preprocessed.csv', index=False)

In [7]:
split_7['Content'] = extract_content_with_progress(split_7['URL'])
split_7.to_csv('split_7_preprocessed.csv', index=False)

Extracting content:   4%|▍         | 75/1797 [01:21<23:04,  1.24it/s]

Error fetching content: 404 Client Error: Not Found for url: https://www.cbc.ca/5-medical-costs-you-may-have-to-pay-for-1.2945539?cmp=rss


Extracting content:   5%|▍         | 88/1797 [01:33<20:53,  1.36it/s]

Error fetching content: 404 Client Error: Not Found for url: https://www.cbc.ca/regional-psychiatric-centre-could-face-eviction-under-saskatchewan-laws-1.2940777?cmp=rss


Extracting content:   6%|▋         | 113/1797 [01:57<30:50,  1.10s/it]

...

Extracting content:  95%|█████████▍| 1704/1797 [29:35<01:39,  1.07s/it]

Error fetching content: Failed to parse: http://…


Extracting content:  95%|█████████▌| 1711/1797 [29:41<01:12,  1.19it/s]

Error fetching content: 404 Client Error: Not Found for url: https://www.who.int/mediacentre/news/releases/2014/hepatitis-guidelines/en/


Extracting content: 100%|██████████| 1797/1797 [31:06<00:00,  1.04s/it]


In [7]:
split_8['Content'] = extract_content_with_progress(split_8['URL'])
split_8.to_csv('split_8_preprocessed.csv', index=False)

Extracting content:   5%|▌         | 96/1797 [01:42<23:26,  1.21it/s]

Error fetching content: 404 Client Error: Not Found for url: https://www.cbc.ca/newsblogs/yourcommunity/2014/02/live-chat-youth-obsession-with-technology.html


Extracting content:   7%|▋         | 124/1797 [02:14<17:30,  1.59it/s]

Error fetching content: 404 Client Error: Not Found for url: https://t.co/eUft%E2%80%A6


Extracting content:  12%|█▏        | 209/1797 [03:47<20:03,  1.32it/s]

...

Extracting content:  90%|█████████ | 1622/1797 [29:10<01:19,  2.19it/s]

Error fetching content: 404 Client Error: Not Found for url: https://t.co/kNe8Pa%E2%80%A6


Extracting content: 100%|█████████▉| 1795/1797 [30:57<00:00,  2.14it/s]

Error fetching content: Failed to parse: http://t…


Extracting content: 100%|██████████| 1797/1797 [30:58<00:00,  1.03s/it]


In [8]:
split_9['Content'] = extract_content_with_progress(split_9['URL'])
split_9.to_csv('split_9_preprocessed.csv', index=False)

Extracting content:   0%|          | 5/1797 [00:03<17:26,  1.71it/s]

Error fetching content: 404 Client Error: Not Found for url: https://t.co/x%E2%80%A6


Extracting content:   1%|          | 10/1797 [00:09<24:45,  1.20it/s]

Error fetching content: 404 Client Error: Not Found for url: https://t.co/T5g%E2%80%A6


Extracting content:   1%|          | 18/1797 [00:12<14:48,  2.00it/s]

...

Error fetching content: HTTPConnectionPool(host='lati.ms', port=80): Max retries exceeded with url: /lFP2R (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7c728a8fa530>, 'Connection to lati.ms timed out. (connect timeout=4)'))


Extracting content: 100%|█████████▉| 1796/1797 [53:50<00:02,  2.87s/it]

Error fetching content: HTTPConnectionPool(host='lati.ms', port=80): Max retries exceeded with url: /lFU8z (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7c728a2ae2c0>, 'Connection to lati.ms timed out. (connect timeout=4)'))


Extracting content: 100%|██████████| 1797/1797 [53:54<00:00,  1.80s/it]

Error fetching content: HTTPConnectionPool(host='lati.ms', port=80): Max retries exceeded with url: /lFQf1 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7c728a2aec20>, 'Connection to lati.ms timed out. (connect timeout=4)'))


In [9]:
split_10['Content'] = extract_content_with_progress(split_10['URL'])
split_10.to_csv('split_10_preprocessed.csv', index=False)

Extracting content:   0%|          | 1/1799 [00:04<2:00:32,  4.02s/it]

Error fetching content: HTTPConnectionPool(host='lati.ms', port=80): Max retries exceeded with url: /lFOEJ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7c728a2af5e0>, 'Connection to lati.ms timed out. (connect timeout=4)'))


Extracting content:   0%|          | 2/1799 [00:04<53:33,  1.79s/it]  

Error fetching content: 404 Client Error: Not Found for url: https://t.co/%E2%80%A6


Extracting content:   0%|          | 3/1799 [00:08<1:23:54,  2.80s/it]

...

Extracting content:  99%|█████████▉| 1788/1799 [28:20<00:04,  2.50it/s]

Error fetching content: 404 Client Error: Not Found for url: http://feeds.latimes.com/~r/latimes/features/health/~3/ZSh6yiG-jdg/ct-met-hcg-diet-1207-20111207,0,4276774.story
Error fetching content: 404 Client Error: Not Found for url: http://feeds.latimes.com/~r/latimes/features/health/~3/4PEbueqDo4s/sns-rt-us-mode-delivery-maternaltre7b62jf-20111207,0,5345171.story
Error fetching content: 404 Client Error: Not Found for url: http://feeds.latimes.com/~r/latimes/features/health/~3/V871HpDYMLM/la-he-breast-cancer-drugs-20111208,0,5300399.story


Extracting content: 100%|█████████▉| 1791/1799 [28:22<00:02,  2.82it/s]

Error fetching content: 404 Client Error: Not Found for url: http://feeds.latimes.com/~r/latimes/features/health/~3/AYBnh01qkuo/os-carb-diet-cancer-20111209,0,7459472.story
Error fetching content: 404 Client Error: Not Found for url: http://feeds.latimes.com/~r/latimes/features/health/~3/maIYoUqs0po/sns-rt-us-states-rankingtre7b52dp-20111206,0,2739896.story


Extracting content: 100%|██████████| 1799/1799 [28:30<00:00,  1.05it/s]


In [47]:
# List to store DataFrames of all splits
dfs = []

# Read each CSV file and append its DataFrame to the list
for i in range(1, 11):
    filename = f'/content/split_{i}_preprocessed.csv'
    df = pd.read_csv(filename)
    dfs.append(df)

# Concatenate all DataFrames in the list
combined_df = pd.concat(dfs, ignore_index=True)

In [48]:
combined_df['Content'] = combined_df['Content'].apply(lambda x: [x])

In [49]:
# Drop rows with missing content
combined_df.dropna(subset=['Content'], inplace=True)

# Combine content paragraphs into a single string
combined_df['Content'] = [" ".join([str(p_tag).strip() for p_tag in content]) for content in combined_df['Content']]

In [52]:
combined_df.tail()

,ID,DateTime,Article Title and Link,URL,Content
17967,144952345170690052,Fri Dec 09 01:31:46 +0000 2011,"Shift work: Good for your pocketbook, bad for ...",http://lat.ms/srI28g,these days its just good to have a job but rem...
17968,144936916838649857,Fri Dec 09 00:30:28 +0000 2011,Hold the pasta: Starchy food linked to breast ...,http://lat.ms/vopkPz,this post has been corrected see note at botto...
17969,144917037414756354,Thu Dec 08 23:11:28 +0000 2011,"Chronic pain is common child complaint, study ...",http://lat.ms/sqbbYO,chronic pain is usually thought of as a proble...
17970,144881597869592576,Thu Dec 08 20:50:39 +0000 2011,"Sporadic low-carb diets may help lower weight,...",http://lat.ms/vz4beE,an intermittent low carb diet could be better ...
17971,144861369597050880,Thu Dec 08 19:30:16 +0000 2011,"O.B. Ultra tampons are coming back, and the co...",http://lat.ms/tu6n9S,women rejoice your ob ultra tampons are on the...


In [51]:
# Function to clean text
def clean_text(text):
    # Remove HTML tags
    text = BeautifulSoup(text, 'html.parser').get_text()
    # Substitute hyphens with empty spaces
    text = re.sub(r'-', ' ', text)
    # Remove non-alphabetic characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Convert text to lowercase
    text = text.lower()
    return text

# Apply the clean_text function to the 'Content' column
combined_df['Content'] = combined_df['Content'].apply(clean_text)

In [53]:
# Convert to wordnet tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None  # Use default POS for lemmatization

In [54]:
# Tokenize and preprocess text data
def preprocess_text(text):
    tokens = word_tokenize(text)  # Tokenize text
    lemmatizer = WordNetLemmatizer()  # Initialize lemmatizer
    pos_tags = nltk.pos_tag(tokens)  # Get part-of-speech tags
    for i, (token, tag) in enumerate(pos_tags):
        pos = get_wordnet_pos(tag)  # Convert NLTK POS tags to WordNet POS tags
        if pos:
            tokens[i] = lemmatizer.lemmatize(token, pos=pos)  # Lemmatize tokens
        else:
            tokens[i] = lemmatizer.lemmatize(token)  # Use default POS for lemmatization
    stop_words = set(stopwords.words('english'))  # Get stopwords
    custom_stopwords = [    # Custom stopwords
    "patient", "doctor", "say", "year", "state", "day", "need", "come", "well",
    "make", "think", "know", "go", "use", "one", "like", "people", "may",
    "many", "still", "even", "two", "way", "good", "much", "back", "new",
    "time", "first", "really",
    "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n",
    "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z"
    ]
    tokens = [token for token in tokens if token not in stop_words and token not in custom_stopwords]  # Remove stopwords
    return ' '.join(tokens)

combined_df['Content'] = combined_df['Content'].apply(preprocess_text)

In [56]:
combined_df.iloc[1]['URL']

'http://www.cbc.ca/news/health/sabra-hummus-recalled-in-u-s-1.3026865?cmp=rss'

In [10]:
# Save preprocessed data to CSV
combined_df.to_csv('combined_preprocessed.csv', index=False)